<a href="https://colab.research.google.com/github/abiodun6948/Predictive-Control-Framework-for-Energy-Water-Desalination-Testbed/blob/main/Copy_of_Untitled4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

np.random.seed(42)

timesteps = 1000

# Control Inputs
pump_speed = 50 + 10*np.sin(np.linspace(0, 20, timesteps)) + np.random.randn(timesteps)
valve_pos = 50 + 5*np.sin(np.linspace(0, 10, timesteps)) + np.random.randn(timesteps)
energy = 5 + 0.5*np.sin(np.linspace(0, 15, timesteps)) + 0.1*np.random.randn(timesteps)

# System States (outputs) - simulate some dynamics
flow = 0.5*pump_speed + 0.2*valve_pos + 0.1*energy + np.random.randn(timesteps)
salinity = 300 + 0.1*energy - 0.05*pump_speed + 0.02*valve_pos + 2*np.random.randn(timesteps)
pressure = 1 + 0.05*pump_speed + 0.03*flow + 0.01*valve_pos + 0.1*np.random.randn(timesteps)

# Create DataFrame
data = pd.DataFrame({
    'pump_speed': pump_speed,
    'valve_pos': valve_pos,
    'energy': energy,
    'flow': flow,
    'salinity': salinity,
    'pressure': pressure
})

print(data.head())


   pump_speed  valve_pos    energy       flow    salinity  pressure
0   50.496714  51.399355  4.932482  34.113669  297.269412  5.019863
1   50.061923  50.974683  4.993055  34.864818  298.953296  5.013446
2   51.047982  50.159724  4.935771  35.635907  298.980406  4.943509
3   52.123269  49.503191  4.991719  38.349132  299.828333  5.218660
4   50.565792  50.898370  4.840651  36.503188  296.240026  5.205652


In [ ]:
from sklearn.preprocessing import MinMaxScaler

# Normalize data
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data)

# Create sequences
def create_sequences(data, seq_length=10):
    X, y = [], []
    for i in range(len(data)-seq_length):
        X.append(data[i:i+seq_length, 0:3])  # Inputs: pump_speed, valve_pos, energy
        y.append(data[i+seq_length, 3:])     # Outputs: flow, salinity, pressure
    return np.array(X), np.array(y)

seq_length = 10
X, y = create_sequences(scaled_data, seq_length)
print(X.shape, y.shape)  # (samples, seq_length, input_features), (samples, output_features)


(990, 10, 3) (990, 3)


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

model = Sequential()
model.add(LSTM(64, input_shape=(seq_length, 3), activation='tanh'))
model.add(Dense(3))  # Predict flow, salinity, pressure
model.compile(optimizer='adam', loss='mse')

model.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 64)             │        17,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 3)              │           195 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 17,603 (68.76 KB)

 Trainable params: 17,603 (68.76 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
history = model.fit(X, y, epochs=50, batch_size=32, validation_split=0.2)

Epoch 1/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 20ms/step - loss: 0.1001 - val_loss: 0.0157
Epoch 2/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0160 - val_loss: 0.0119
Epoch 3/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.0122 - val_loss: 0.0117
Epoch 4/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0110 - val_loss: 0.0118
Epoch 5/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0111 - val_loss: 0.0116
Epoch 6/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0110 - val_loss: 0.0118
Epoch 7/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0106 - val_loss: 0.0120
Epoch 8/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.0102 - val_loss: 0.0116
Epoch 9/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.0099 - val_loss: 0.0117
Epoch 10/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.0111 - val_loss: 0.0115
Epoch 11/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: 0.0109 - val_loss: 0.0117
Epoch 12/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.010

In [ ]:
# Predict next 5 steps (example)
pred_horizon = 5
input_seq = X[-1:]  # last sequence
predictions = []
current_seq = input_seq.copy()

for _ in range(pred_horizon):
    y_pred = model.predict(current_seq)
    predictions.append(y_pred[0])
    # Append predicted output as next step with dummy input (you may use planned inputs)
    next_input = current_seq[0, 1:, :]  # shift left
    next_input = np.vstack([next_input, [[0.5, 0.5, 0.5]]])  # example: planned inputs normalized
    current_seq[0] = next_input

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 898ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
